# Graph coloring

In [2]:
import numpy as np
import os
from collections import OrderedDict

### Loading data

In [308]:
def parse_items(input_data):
    
    lines = input_data.strip().split('\n')
    items = []
    
    for i,line in enumerate(lines):
        line = line.split()
        if i==0: 
            #1st line is header
            num_nodes, num_edges = map(int, line)
        else:
            items.append(list(map(int, line)))

    return num_nodes, num_edges, items

data_dir = '../data/'
file_name = 'gc_70_7'

with open(os.path.join(data_dir, file_name), 'r') as input_data_file:
            input_data = input_data_file.read()
num_nodes, num_edges, items = parse_items(input_data)
print('num_nodes =', num_nodes)
# print(items)


num_nodes = 70


### Welsh-Powell algorithm

In [233]:
# testing use case
# http://mrsleblancsmath.pbworks.com/w/file/fetch/46119304/vertex%20coloring%20algorithm.pdf
# g = {
#     0: [1,2],
#     1: [0,5,6,7,8],
#     2: [0,3],
#     3: [2,4,5,7],
#     4: [3], 
#     5: [1,6], 
#     6: [1,5,7], 
#     7: [1,6,8,10], 
#     8: [1,7,9], 
#     9: [8,10], 
#     10: [7,9]
# }
# g = OrderedDict(sorted(g.items(), key= lambda x: len(x[1]), reverse=True))


In [234]:
def make_ordered_graph(items,num_nodes):

    g = {i:set() for i in range(num_nodes)}
    for u,v in items:
        g[u].add(v)
        g[v].add(u)

    # sort graph by valence of nodes
    g = OrderedDict(sorted(g.items(), key= lambda x: len(x[1]), reverse=True))

    return g
    
def algo_heuristic(g):
    # Welsh-Powell algorithm http://mrsleblancsmath.pbworks.com/w/file/fetch/46119304/vertex%20coloring%20algorithm.pdf

    colors = [None]*len(g)
    color = 0
    n_init, _ = g.popitem(last=False)
    colors[n_init] = color

    while None in colors:
        for u, nbrs in g.items():
            if colors[u] is None:
                nbrs_colors = [colors[v] for v in nbrs]
                if color not in nbrs_colors:
                    # print(f'assigned {color} to {u}')
                    colors[u] = color
        color += 1

    return colors

### Constraint Programming Solver

In [237]:
from ortools.sat.python import cp_model
# http://www.hakank.org/google_cp_solver/coloring_cp_sat.py
# https://www.xiang.dev/cp-sat/

In [305]:
data_dir = '../data/'
file_name = 'gc_250_9'

with open(os.path.join(data_dir, file_name), 'r') as input_data_file:
            input_data = input_data_file.read()
num_nodes, num_edges, items = parse_items(input_data)
print('num_nodes =', num_nodes)
# print(edges)

num_nodes = 250


In [296]:
def algo_cp(items, num_nodes, target, time_limit=10):

    # Creates the solver.
    model = cp_model.CpModel()

    # Creates the variables.
    # colors array
    colors = [
        model.NewIntVar(0, num_nodes - 1, 'c%i' % i) for i in range(num_nodes)
    ]
    # max color in array
    max_color = model.NewIntVar(0, num_nodes - 1 ,'obj')

    # Creates the constraints.
    # colors of nodes connected by edges must be different
    for u,v in items:
        model.Add(colors[u] != colors[v])
    # define max_color as the max of values in colors.
    model.AddMaxEquality(max_color, colors)
    # Symmetry breaking: impose simple precendence rule on values of colors[i]
    model.Add(colors[0] == 0)
    for i in range(num_nodes):
        model.Add(colors[i] <= i+1)

    # Minimize objective: model.Minimize(max_color) is too slow so we fix the 
    # target and solve for feasibility
    model.Add(max_color <= target)

    # Solve and print out the solution.
    solver = cp_model.CpSolver()
    # Sets a time limit.
    solver.parameters.max_time_in_seconds = time_limit
    # Solve the model
    solver.Solve(model)
    # if solvers run over time limit return None
    try:
        results = [solver.Value(colors[i]) for i in range(num_nodes)]
    except IndexError:
        results = None

    return results



In [307]:
results = algo_cp(items, num_nodes, 94, 600)
print(results)

[0, 2, 3, 1, 4, 1, 5, 8, 9, 10, 7, 5, 6, 0, 15, 14, 14, 18, 19, 11, 12, 16, 23, 17, 23, 25, 21, 28, 22, 18, 10, 8, 25, 2, 29, 19, 32, 37, 31, 26, 4, 33, 24, 44, 37, 38, 41, 45, 45, 34, 50, 2, 20, 51, 48, 22, 49, 32, 19, 57, 56, 46, 50, 62, 53, 60, 42, 59, 65, 43, 52, 70, 26, 69, 60, 68, 9, 61, 38, 74, 43, 66, 83, 79, 74, 7, 42, 77, 61, 34, 86, 58, 75, 47, 44, 0, 70, 60, 59, 79, 93, 87, 78, 82, 82, 35, 72, 54, 17, 85, 29, 93, 48, 68, 11, 81, 55, 3, 89, 8, 76, 38, 25, 91, 69, 84, 30, 92, 53, 58, 73, 80, 71, 64, 4, 36, 36, 87, 5, 46, 67, 13, 64, 15, 75, 14, 39, 94, 90, 90, 73, 88, 15, 47, 3, 40, 27, 94, 93, 92, 91, 90, 85, 89, 88, 87, 86, 85, 84, 83, 82, 81, 80, 79, 78, 77, 76, 75, 74, 73, 72, 71, 70, 69, 68, 67, 66, 65, 55, 64, 63, 62, 61, 60, 59, 58, 57, 56, 55, 54, 53, 52, 51, 50, 49, 48, 47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30, 29, 1, 28, 27, 30, 26, 24, 22, 21, 20, 17, 16, 20, 86, 63, 92, 13, 12, 72, 6, 27, 43, 39, 14, 7, 6]


### Solution ubmission

In [304]:
def solver_submission(input_data):

    num_nodes, num_edges, items = parse_items(input_data)

    # use heuristic to establish higher bound 
    g = make_ordered_graph(items,num_nodes)
    colors = algo_heuristic(g)

    # iteratively improve the results with cp 
    target = max(colors)
    while True:
        new_colors = algo_cp(items, num_nodes, target)
        if new_colors is not None:
            colors = new_colors
            target -= 1
        else:
            break

    n_colors = max(colors)+1
    optimal_solution = 0

    return f'{n_colors } {optimal_solution}\n{" ".join(map(str, colors))}'

data_dir = '../data/'
file_name = 'gc_70_7'

with open(os.path.join(data_dir, file_name), 'r') as input_data_file:
            input_data = input_data_file.read()
with open('solution', 'w') as f:
    f.write(solver_submission(input_data))